In [1]:
# importation
import numpy as np
from torch.autograd import Variable
import torch
import matplotlib.pyplot as plt
%matplotlib inline
# local
from FBResNet.myfunc import Physics
from FBResNet.myfunc import MyMatmul
from FBResNet.main import FBRestNet
from FBResNet.model import MyModel
from FBResNet.posttreat import Export_hyper

### Parametres - experimental conditions

In [2]:
exp1 = FBRestNet(experimentation=Physics(2000,50,0.5,1), constraint = 'cube',\
                 nb_blocks=20, noise = 0.05,\
                 folder = './', im_set="Set1",batch_size=[600,1],\
                 lr=1e-3, nb_epochs=[30,1])

### Synthetic dataset

In [3]:
train_set, val_set = exp1.CreateDataSet()

### Training

In [ ]:
exp1.train(train_set,val_set,save_model=True)

epoch :  0  -----  validation :  0.0001410258924400081
epoch :  1  -----  validation :  0.00011453772680598473
epoch :  2  -----  validation :  9.079562721295524e-05
epoch :  3  -----  validation :  7.225494096476279e-05
epoch :  4  -----  validation :  5.68045240205304e-05
epoch :  5  -----  validation :  4.4321027070824794e-05
epoch :  6  -----  validation :  3.452297847277933e-05
epoch :  7  -----  validation :  2.8105821399718414e-05


### Gaussian test

In [ ]:
exp1.test_gauss()

### Lipschitz constant

In [ ]:
exp1.model.Lipschitz(opt1="semi",opt2="entree11")

In [ ]:
exp1.model.Lipschitz(opt1="semi",opt2="entree01")